## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Adrar,MR,20.5022,-10.0711,101.73,9,60,9.28,broken clouds
1,1,Kaitangata,NZ,-46.2817,169.8464,46.04,85,84,5.68,broken clouds
2,2,Gizo,SB,-8.1030,156.8419,81.59,77,40,6.22,scattered clouds
3,3,Sakakah,SA,29.9697,40.2064,91.54,16,22,3.98,few clouds
4,4,Parfenyevo,RU,58.4840,43.4088,58.48,80,99,2.28,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 60
What is the maximun temperature you would like for your trip? 100


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Gizo,SB,-8.1030,156.8419,81.59,77,40,6.22,scattered clouds
3,3,Sakakah,SA,29.9697,40.2064,91.54,16,22,3.98,few clouds
5,5,Rikitea,PF,-23.1203,-134.9692,73.65,81,86,14.65,moderate rain
6,6,Puerto Carreno,CO,6.1890,-67.4859,80.69,84,73,5.30,broken clouds
7,7,Termiz,UZ,37.2242,67.2783,82.49,28,0,2.30,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

480


City_ID                480
City                   480
Country                476
Lat                    480
Lng                    480
Max Temp               480
Humidity               480
Cloudiness             480
Wind Speed             480
Current Description    480
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

476


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Gizo,SB,-8.1030,156.8419,81.59,77,40,6.22,scattered clouds
3,3,Sakakah,SA,29.9697,40.2064,91.54,16,22,3.98,few clouds
5,5,Rikitea,PF,-23.1203,-134.9692,73.65,81,86,14.65,moderate rain
6,6,Puerto Carreno,CO,6.1890,-67.4859,80.69,84,73,5.30,broken clouds
7,7,Termiz,UZ,37.2242,67.2783,82.49,28,0,2.30,clear sky


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Gizo,SB,81.59,scattered clouds,-8.1030,156.8419,
3,Sakakah,SA,91.54,few clouds,29.9697,40.2064,
5,Rikitea,PF,73.65,moderate rain,-23.1203,-134.9692,
6,Puerto Carreno,CO,80.69,broken clouds,6.1890,-67.4859,
7,Termiz,UZ,82.49,clear sky,37.2242,67.2783,
8,Yellowknife,CA,66.72,overcast clouds,62.4560,-114.3525,
9,Vaini,TO,75.40,scattered clouds,-21.2000,-175.2000,
12,Mataura,PF,73.65,moderate rain,-23.1203,-134.9692,
14,Botwood,CA,66.72,overcast clouds,62.4560,-114.3525,
15,Hamilton,US,74.17,moderate rain,39.1834,-84.5333,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Gizo,SB,81.59,scattered clouds,-8.1030,156.8419,Imagination Island
3,Sakakah,SA,91.54,few clouds,29.9697,40.2064,Fakhamat Al Orjoana Apartment
5,Rikitea,PF,73.65,moderate rain,-23.1203,-134.9692,People ThankYou
6,Puerto Carreno,CO,80.69,broken clouds,6.1890,-67.4859,Casa Hotel Victoria
7,Termiz,UZ,82.49,clear sky,37.2242,67.2783,Hotel Sharq
8,Yellowknife,CA,66.72,overcast clouds,62.4560,-114.3525,The Explorer Hotel
9,Vaini,TO,75.40,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Mataura,PF,73.65,moderate rain,-23.1203,-134.9692,People ThankYou
14,Botwood,CA,66.72,overcast clouds,62.4560,-114.3525,The Explorer Hotel
15,Hamilton,US,74.17,moderate rain,39.1834,-84.5333,Six Acres Bed & Breakfast


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))